In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt
from utils import *
import pickle

In [ ]:
class KibaModel(torch.nn.Module):
    def __init__(self, input_size):
        super(KibaModel, self).__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 256),
            torch.nn.BatchNorm1d(256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.4),
            torch.nn.Linear(256, 128),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(128, 64),
            torch.nn.BatchNorm1d(64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1)
        )
    
    def forward(self, x):
        return self.model(x)

In [ ]:
# Load the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size = 576 # This assumes the same feature engineering has happened to the test set
model = KibaModel(input_size)
model.load_state_dict(torch.load('regression_model.pth')) 
model.to(device)
model.eval()

In [ ]:
# Prepare scaler (use the one fitted during training)
scaler = StandardScaler()
scaler_path = "./scaler.pkl"
with open(scaler_path, "rb") as f:
    scaler = pickle.load(f)

In [ ]:
# Inference pipeline
def predict_score(uniprot_id, pubchem_cid):
    # Generate embeddings
    try:
        protein_embedding = get_protein_embedding(uniprot_id)
    except Exception as e:
        print(f"Error generating protein embedding: {e}")
        return None

    chemical_embedding = generate_random_projection(pubchem_cid)

    # Combine features and scale
    combined_features = np.concatenate((protein_embedding, chemical_embedding)).reshape(1, -1)
    scaled_features = scaler.transform(combined_features)

    # Convert to PyTorch tensor
    input_tensor = torch.tensor(scaled_features, dtype=torch.float32).to(device)

    # Make prediction
    with torch.no_grad():
        output = model(input_tensor)
        prediction = outputs.cpu().numpy()
        prediction = np.expm1(prediction)

    return prediction

In [ ]:
# Example usage
uniprot_id = "P12345"  # an example of UniProt ID
pubchem_cid = "123456"  # an example of PubChem CID
prediction = predict_score(uniprot_id, pubchem_cid)
if prediction is not None:
    print(f"Predicted kiba score: {prediction:.4f}")